## Pandas批量拆分Excel与合并Excel

实例演示：
1. 将一个大Excel等份拆成多个Excel
2. 将多个小Excel合并成一个大Excel并标记来源

In [27]:
work_dir="./course_datas/c15_excel_split_merge"
splits_dir=f"{work_dir}/splits"

import os
if not os.path.exists(splits_dir):
    os.mkdir(splits_dir)

### 0、读取源Excel到Pandas

In [28]:
import pandas as pd

In [29]:
df_source = pd.read_excel(f"{work_dir}/crazyant_blog_articles_source.xlsx", engine='openpyxl')

In [30]:
df_source.head()

,id,title,tags
0,2585,Tensorflow怎样接收变长列表特征,"python,tensorflow,特征工程"
1,2583,Pandas实现数据的合并concat,"pandas,python,数据分析"
2,2574,Pandas的Index索引有什么用途？,"pandas,python,数据分析"
3,2564,机器学习常用数据集大全,"python,机器学习"
4,2561,一个数据科学家的修炼路径,数据分析


In [31]:
df_source.index

RangeIndex(start=0, stop=258, step=1)

In [32]:
df_source.shape

(258, 3)

In [33]:
total_row_count = df_source.shape[0]
total_row_count

258

### 一、将一个大Excel等份拆成多个Excel

1. 使用df.iloc方法，将一个大的dataframe，拆分成多个小dataframe
2. 将使用dataframe.to_excel保存每个小Excel

#### 1、计算拆分后的每个excel的行数

In [34]:
# 这个大excel，会拆分给这几个人
user_names = ["xiao_shuai", "xiao_wang", "xiao_ming", "xiao_lei", "xiao_bo"]

In [35]:
# 每个人的任务数目
split_size = total_row_count // len(user_names)
if total_row_count % len(user_names) != 0:
    split_size += 1

split_size

52

#### 2、拆分成多个dataframe

In [40]:
pd.__version__

'1.1.5'

In [39]:
df_subs = []
for idx, user_name in enumerate(user_names):
    # iloc的开始索引
    begin = idx*split_size
    # iloc的结束索引
    end = begin+split_size
    # 实现df按照iloc拆分
    print(begin, end)
    df_sub = df_source.iloc[begin:end]
    print(df_sub.size)
    # 将每个子df存入列表
    df_subs.append((idx, user_name, df_sub))

0 52
156
52 104
156
104 156
156
156 208
156
208 260
150


#### 3、将每个datafame存入excel

In [37]:
for idx, user_name, df_sub in df_subs:
    file_name = f"{splits_dir}/crazyant_blog_articles_{idx}_{user_name}.xlsx"
    df_sub.to_excel(file_name, index=False)

### 二、合并多个小Excel到一个大Excel

1. 遍历文件夹，得到要合并的Excel文件列表
2. 分别读取到dataframe，给每个df添加一列用于标记来源
3. 使用pd.concat进行df批量合并
4. 将合并后的dataframe输出到excel

#### 1. 遍历文件夹，得到要合并的Excel名称列表

In [ ]:
import os
excel_names = []
for excel_name in os.listdir(splits_dir):
    excel_names.append(excel_name)
excel_names

#### 2. 分别读取到dataframe

In [ ]:
df_list = []

for excel_name in excel_names:
    # 读取每个excel到df
    excel_path = f"{splits_dir}/{excel_name}"
    df_split = pd.read_excel(excel_path)
    # 得到username
    username = excel_name.replace("crazyant_blog_articles_", "").replace(".xlsx", "")[2:]
    print(excel_name, username)
    # 给每个df添加1列，即用户名字
    df_split["username"] = username
    
    df_list.append(df_split)

#### 3. 使用pd.concat进行合并

In [ ]:
df_merged = pd.concat(df_list)

In [ ]:
df_merged.shape

In [ ]:
df_merged.head()

In [ ]:
df_merged["username"].value_counts()

#### 4. 将合并后的dataframe输出到excel

In [ ]:
df_merged.to_excel(f"{work_dir}/crazyant_blog_articles_merged.xlsx", index=False)